In [9]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [10]:
#učitavanje u DataFrame podaci iz csv datoteke
podaci = pd.read_csv('df.csv')

podaci.drop('name', axis=1, inplace=True)


#ako ne napravimo swap, radimo klasifikaciju po tome je li ili nije packer
key = 'is packer'
keyOut = 'packer type'

#swap ------> radimo klasifikaciju po 'packer type'
#key,keyOut = keyOut,key

#naknadno izbačeni featuresi
podaci.drop('log_max_heap_reserve', axis=1, inplace=True)
podaci.drop('log_repeated_entry_sizes', axis=1, inplace=True)
podaci.drop('non_empty_metadatas', axis=1, inplace=True)
podaci.drop('certificate', axis=1, inplace=True)
podaci.drop('cert_malformed', axis=1, inplace=True)
podaci.drop('rich_header_valid', axis=1, inplace=True)
podaci.drop('max_opt_header_size', axis=1, inplace=True)


names = np.arange(podaci.shape[0])
trening_skup, test_skup, train_ind, test_ind = train_test_split(podaci.drop(keyOut, axis=1),names, test_size=0.34, random_state=0)

print('Number of features:  ', len(list(trening_skup))-1)

Number of features:   52


In [11]:
#oznake za podatke
Y = trening_skup[key]

#podaci
X = trening_skup.drop(key, axis=1)

# Promijenjeni defaultni parametri za RandomForest

|      PARAMETAR          |    IZNOS               | DEFAULT           |
|-------------------------|------------------------|-------------------|
| <b>n_estimators</b>     | <b>800</b>             |  <b>10</b>        |           
| <b>criterion</b>        | <b>entropy</b>         |  <b>gini</b>      |         
| <b>min_samples_split</b>| <b>4</b>               |  <b>2</b>         |                  
| <b>bootstrap</b>        | <b>False</b>           |  <b>True</b>      |          
| <b>n_jobs</b>           | <b>-1</b>              |  <b>1</b>         |          
| <b>random_state</b>     | <b>42</b>              |  <b>random</b>    |          

In [12]:
# Model
rfc = RandomForestClassifier(n_estimators=800, 
                             random_state=42, 
                             criterion = 'entropy',#default='gini' 
                             min_samples_split =4, #default=2
                             bootstrap = False,    #uzorkovanje bez ponavljanja
                              
                             n_jobs=-1, #rad na maksimalnom broju procesora
                            )

# Training
rfc.fit(X, Y);

rezultati = rfc.predict(test_skup.drop(key, axis=1))

broj_testnih_primjeraka = len(test_skup.index)
print('Broj test primjeraka: ' + str(broj_testnih_primjeraka))
broj_tocno_klasificiranih = sum(rezultati == test_skup[key])
print('Broj točno klasificiranih: ' + str(broj_tocno_klasificiranih))
tocnost = broj_tocno_klasificiranih/broj_testnih_primjeraka
print('Točnost: '+str(tocnost*100)+'%')

Broj test primjeraka: 14887
Broj točno klasificiranih: 14819
Točnost: 99.54322563310271%


In [13]:
podaciProba = {'prediction' : rezultati,
              'original' : test_skup[key]}
podaciProba = pd.DataFrame(podaciProba)
tablica = pd.crosstab(podaciProba['original'], podaciProba['prediction'])

tablica

prediction,False,True
original,,
False,7371,18
True,50,7448


In [14]:
if key=='is packer':
   
    podaciProba = {'prediction' : rezultati,
                   'original' : podaci['packer type'][test_ind]}
    podaciProba = pd.DataFrame(podaciProba)
    tablica = pd.crosstab(podaciProba['original'], podaciProba['prediction'])
    
    
else:
    prediction_is_packer = rezultati.copy()
    prediction_is_packer[prediction_is_packer != 't0'] = 'packer'
    prediction_is_packer[prediction_is_packer == 't0'] = 'not packer'
    

    podaciProba = {'prediction' : prediction_is_packer,
                   'original' : podaci['packer type'][test_ind]}
    podaciProba = pd.DataFrame(podaciProba)
    tablica = pd.crosstab(podaciProba['original'], podaciProba['prediction'])

#tablica.plot.bar()
display(tablica)
#postoci po klasama
column1 = tablica[0]/tablica.apply(sum, axis=1) * 100
column2 = tablica[1]/tablica.apply(sum, axis=1) * 100

column3 = column1 + column2

d = {'False': column1, 'True': column2, 'Ukupno': column3}
df = pd.DataFrame(data=d)
print('\n\n' + '\033[1m' + 'Postotci po tipu packera:' + '\033[0m')
display(df)

prediction,False,True
original,,
t0,7371,18
t1,33,640
t2,10,688
t3,5,5013
t4,2,1107




Postotci po tipu packera:


,False,True,Ukupno
original,,,
t0,99.756395,0.243605,100.0
t1,4.903418,95.096582,100.0
t2,1.432665,98.567335,100.0
t3,0.099641,99.900359,100.0
t4,0.180343,99.819657,100.0


In [15]:
r_not_packed = column1[0]/100
#print(r_not_packed)
r_overlay_and_cripters = (tablica[1][1] + tablica[1][2])/(tablica[0][1] + tablica[0][2]+tablica[1][1] + tablica[1][2])
#print(r_overlay_and_cripters)
r_protectors = column2[3]/100
#print(r_protectors)
r_compress = column2[4]/100
#print(r_compress)

bodovi = 8 * r_not_packed**3 * r_overlay_and_cripters * r_protectors * r_compress
print('Bodovi za kvalitetu rjesenja '+str(bodovi))

Bodovi za kvalitetu rjesenja 7.671070313276444
